<a href="https://www.kaggle.com/code/abdalrhamnhebishy/cats-dogs-classification?scriptVersionId=191060770" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
from IPython.core.display import HTML
# Apply styles globally within the notebook
HTML('''
<style>
  h2 {
    font-size: var(--jp-content-font-size4);
    background-color: #FF0000 ;
    color: yellow;
    display: flex;
    align-items: center;
    justify-content: center;
    padding: 10px 41px;
    top: 10px;
    position: relative;
    border-radius: 10px 50px 10px 50px;
}

img {       /* flex-grow: 1; */
            /* flex-shrink: 1; */
            border-radius: 150px 70px 150px 70px;
            border: 10px solid #eee;
            display: flex;
            align-items: center;
            justify-content: center;
            transition: box-shadow 0.3s ease; /* Add a transition for a smooth effect */
        }

       img:active {
          box-shadow: 0 10px 20px rgba(255, 255, 0, 0.5), 0 6px 6px rgba(215, 215, 0, 0.5);
        }
</style>
''')

## Simple Introduce :

* This unstructured Data loaded from Kaggle . 

* I used Deep learning Techniques by using tensorflow pkgs .

* I divide Data into train set & validatio set after loading the data.

* I used ImageDataGenerator for preprocessing the Data .

## About Dataset: 

* The Cats and Dogs Classification dataset is a widely used benchmark dataset in the field of computer vision.

* It consists of thousands of images of cats and dogs, with each image labeled as either a cat or a dog.

* This dataset serves as a fundamental resource for training and evaluating machine learning models for image classification tasks.

* Researchers and practitioners leverage this dataset to develop and test algorithms that can accurately distinguish between images of cats and dogs, contributing to advancements in computer vision technology.





## imports :

In [ ]:
# imports :
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras import models
from tensorflow.keras.layers import Dense , Flatten , Conv2D ,MaxPooling2D
from tensorflow .keras.preprocessing.image import ImageDataGenerator

## preparing & loading the data :

In [ ]:
# Dowanload the data (cat & dogs):
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

In [ ]:
# to unzip the file:
!unzip cats_and_dogs_filtered.zip

In [ ]:
def create_dataset(path):
  images_path=[]
  images_label=[]
  for dir in os.listdir(path):
    for file in os.listdir(os.path.join(path, dir)):
      images_path.append(os.path.join(path, dir, file))
      image_label = 0 if dir=="cats" else 1
      images_label.append(image_label)
  return images_path, images_label


In [ ]:
image_train,label_train=create_dataset("cats_and_dogs_filtered/train")
image_test,label_test=create_dataset("cats_and_dogs_filtered/validation")
print(f"image_train_shape:{np.shape(image_train)}\n label_train_shape:{np.shape(label_train)}")
print(f"image_test_shape:{np.shape(image_test)}\n label_test_shape:{np.shape(label_test)}")

In [ ]:
image_train[1:10]

In [ ]:
label_train[2]

##  Agumention :

In [ ]:
# function that read the image & conver into jpeg:

def get_image_tensor_from_path(image_path,label):
    image=tf.io.read_file(image_path)
    image=tf.image.decode_jpeg(image)
    image=tf.image.convert_image_dtype(image,tf.float32)
    image=tf.image.resize(image,(244,244))


    return image , label

In [ ]:
# function to change from brightness or reloation of the image :
def agument(image,label):
    image=tf.image.random_brightness(image,max_delta=32.0/255.0) # change from brightness of image

    image=tf.image.random_flip_left_right(image) # change from position of image

    image=tf.image.random_flip_up_down(image)

    image=tf.image.random_saturation(image,lower=500000,upper=1000000)      # to change preentage of color in image


    return image , label

In [ ]:
def cd_dataset(x,y,batch_size=32,training=False):
    data=tf.data.Dataset.from_tensor_slices((x,y))

    data=data.map(get_image_tensor_from_path,num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if training:

        data=data.map(agument,num_parallel_calls=tf.data.experimental.AUTOTUNE)

        data=data.shuffle(2000)

        data=data.batch(batch_size)

        data=data.prefetch(tf.data.experimental.AUTOTUNE)

        return data

In [ ]:
train_dataset=cd_dataset(image_train,label_train,training=True)
test_dataset=cd_dataset(image_test,label_test)

In [ ]:
class_names=["cat","dog"]

for x,y in train_dataset.take(1):

    plt.figure(figsize=(10,10))

    for i in range (25):

        plt.subplot(5,5,i+1)
        plt.imshow(x[i])

        plt.xticks([])
        plt.yticks([])

        plt.xlabel(class_names[y[i]])

## Data  Preprocessing :

In [ ]:
# preprocessing the data :
def reshape_and_normalize(images):
     # reshape :
    images=images.reshape(images.reshape[0],images.reshape[1],images.reshape[2],1)
    
    #Normalize :
    images=images/255
    return images



In [ ]:
# Data Generator :
train_dir = "cats_and_dogs_filtered/train"
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=128,
    class_mode='binary' 
)

validation_dir="cats_and_dogs_filtered/validation"
test_datagen = ImageDataGenerator(rescale=1/255)
test_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    batch_size=128,
    class_mode='binary' 
)


## Bulding Model :

In [ ]:
# Bulding the model :

model=tf.keras.models.Sequential([
    Conv2D(16,(3,3),activation="relu",input_shape=(300,300,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512,activation="relu"),
    Dense(1,activation="sigmoid"),
        ])
model.summary()

In [ ]:
# Callback function:

class mycallback(tf.keras.callbacks.Callback):

    def one_epoch_end(self,epoch,logs={}):
        if(logs.get("accuracy")>0.995):
            print("/n Stop trining :")
            self.model.stop_training=True

In [ ]:
# RMSprop optimizer:

from tensorflow.keras.optimizers import RMSprop

model.compile(
    
    loss='binary_crossentropy',
    optimizer=RMSprop(lr=0.001),  
    metrics=["accuracy"]
)

callbacks=mycallback()

# Training the model:

print("\n Model Training :")

history=model.fit(
train_generator,
steps_per_epoch=8,
epochs=50,
validation_data=test_generator,
validation_steps=8,
verbose=2,callbacks=[callbacks])


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

# Assuming `model` is your trained model
# Load the model here if it's not already loaded

# Get the image path from the user
path = input("Enter image path: ")

# Check if the file exists
if not os.path.isfile(path):
    print("The specified file does not exist.")
else:
    # Load the image
    img = load_img(path, target_size=(300, 300))
    x = img_to_array(img)
    x /= 255.0
    
    # Expand dimensions to match the model's input shape
    x = np.expand_dims(x, axis=0)
    
    # Predict the class
    classes = model.predict(x, batch_size=1)
    
    # Print the predicted class
    print(classes[0])
    if classes[0] > 0.5:
        print(f"{os.path.basename(path)} is a dog")
    else:
        print(f"{os.path.basename(path)} is a cat")

    # Display the image
    plt.imshow(img)
    plt.title(f"Prediction: {'Dog' if classes[0] > 0.5 else 'Cat'}")
    plt.axis('off')
    plt.show()

In [ ]:
# Evalute on the test set:

print("\n Model Evaluation :")
test_loss = model.evaluate(test_generator, verbose=2)
print(test_loss)

In [ ]:
print("\n Model Prediction :")
y_predict=model.predict(test_generator)
y_predict[:10]


In [ ]:
# compute min & max in the y_prediction:
print(f"the min_value for prediction --->> {np.min(y_predict)} \n the max_value for prediction --->>> {np.max(y_predict)} ")

In [ ]:
# Plotting training history (accuracy and loss)
plt.figure(figsize=(12, 6))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')

plt.tight_layout()
plt.show()

## End project 

* Devloper : AbdalRahman Gameel Ahmed Hebishy

* if you find my Notebook useful upvote me & Leave a comment if you have any Question

* Kaggle works : https://www.kaggle.com/work
   